In [2]:
from helperFunctions import *
from constants import *

In [1]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
import numpy

In [3]:
from filterDataScript import *

In [4]:
filename2 = "part_P002_block_S001"
XDF_Path2 = "./data/"+filename2+".xdf"
XDF_Data2 = loadxdf(XDF_Path2)

filename3 = "part_P003_block_S001"
XDF_Path3 = "./data/"+filename3+".xdf"
XDF_Data3 = loadxdf(XDF_Path3)

filename1 = "part_P001_block_S001"
XDF_Path1 = "./data/"+filename1+".xdf"
XDF_Data1 = loadxdf(XDF_Path1)

In [13]:
trial1 = filterDataForPipline(XDF_Data1)
trial2 = filterDataForPipline(XDF_Data2)
trial3 = filterDataForPipline(XDF_Data3)


In [7]:
model = LinearDiscriminantAnalysis()

In [43]:
XTrial1 = []
YTrial1 = []
for i in trial1:
    XTrial1.append(i[1])
    YTrial1.append(i[0])

XTrial2 = []
YTrial2 = []
for i in trial2:
    XTrial2.append(i[1])
    YTrial2.append(i[0])

XTrial3 = []
YTrial3 = []
for i in trial3:
    XTrial3.append(i[1])
    YTrial3.append(i[0])

In [65]:
XTrial1Avg = []
for i in range(len(XTrial1)):
    XTrial1Avg.append(np.mean(XTrial1[i], axis = 0))

XTrial2Avg = []
for i in range(len(XTrial2)):
    XTrial2Avg.append(np.mean(XTrial2[i], axis = 0))


XTrial3Avg = []
for i in range(len(XTrial3)):
    XTrial3Avg.append(np.mean(XTrial3[i], axis = 0))


In [66]:
# XTrial1 = np.zeros((10, 4))
# YTrial1 = numpy.zeros((10))

# idx = 0;
# for i in trial1:
#     XTrial1[idx].append(i[1])
#     YTrial1[idx] = i[0]
#     idx += 1

# idx = 0;
# XTrial2 = numpy.zeros((20, 4))
# YTrial2 = numpy.zeros((10))
# for i in trial2:
#     XTrial2[idx] = (i[1])
#     YTrial2[idx] = (i[0])
#     idx += 1

# idx = 0;
# XTrial3 = numpy.zeros((20, 4))
# YTrial3 = numpy.zeros((10))
# for i in trial3:
#     XTrial3[idx] = (i[1])
#     YTrial3[idx] = (i[0])
#     idx += 1

In [68]:
XTrain = XTrial1Avg + XTrial2Avg
YTrain = YTrial1 + YTrial2



In [74]:
model.fit(XTrain, YTrain)

/Users/Alvin/anaconda3/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [70]:
YPred= model.predict(XTrial3Avg)

In [75]:
from sklearn.metrics import accuracy_score

In [76]:
accuracy_score(YTrial3, YPred)

0.5